# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

**I built a coding expert tutor with 2 models: Gemini and GPT.
It works with streamining and tools simultaneously.
If a user asks a mathematical question, the Dalle 3 will generate an image of that equation.**

In [ ]:
import gradio
from openai import OpenAI
import os
from dotenv import load_dotenv
import math
import json
import base64
from io import BytesIO
from PIL import Image

In [ ]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")
    
 
GPT_MODEL = "gpt-5-nano"
GEMINI_MODEL = "gemini-2.5-flash"
openai = OpenAI()
gemini = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)
   

In [ ]:
system_message = "You are an expert coding tutor. \n" \
"You explain the answers in a friendly and easy to understand way.\n" \
"However, if the input from the user feels too vague, ask them to provide more details before answering."

In [ ]:
def calculate_math(math_equation):
    print("Math calculator tool has been run...")
    
    allowed = {"__builtins__": None}
    allowed.update({k: getattr(math, k) for k in dir(math) if not k.startswith("_")})
    
    result = eval(math_equation, allowed, {})
    return result
            

In [ ]:
calculate_math("sqrt(25)")

In [ ]:
calculate_math_function = {
    "name": "calculate_math",
    "description": "Calculate math requested by the user. You should run this tool when a user asks to know the result of ANY equation. For example: 'What is ther result of this: sqrt(25)'",
    "parameters": {
        "type": "object",
        "properties": {
            "math_equation": {
                "type": "string",
                "description": "The math question the user wants to calculate. You should pass only the math equation, not text. For example: sqrt(25)",
            },
        },
        "required": ["math_equation"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": calculate_math_function}]

In [ ]:
def generate_math_result_image(equation, result):
    image_response = openai.images.generate(
        model="dall-e-3",
        prompt=f"Generate a realistic image of a math equation: '{equation}={result}' on a school chalk board with.",
        size="1024x1024",
        n=1,
        response_format="b64_json",
    )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [ ]:
def chat(history, model="GPT"):
    messages = [{"role": "system", "content": system_message}] + history
    if model == "GPT":   
        response = openai.chat.completions.create(model=GPT_MODEL, messages=messages, stream=True, tools=tools)
    else:
        response = gemini.chat.completions.create(model=GEMINI_MODEL, messages=messages, stream=True, tools=tools)
    
    buffer = {"role": "assistant", "content": "", "tool_calls": []}
    tool_answer = ""
    image = None
    
    for chunk in response:
        delta = chunk.choices[0].delta
        if delta.content:
            buffer["content"] += delta.content or ""
            yield history + [buffer], image

        if delta.tool_calls:
            if delta.tool_calls[0].function.name:
                buffer["tool_calls"].append(delta.tool_calls[0])
            for call in delta.tool_calls:
                if call.function and model == "GPT":
                    buffer["tool_calls"][0].function.arguments += call.function.arguments
        
        if chunk.choices[0].finish_reason == "tool_calls":
            tool_call = buffer["tool_calls"][0]
            response, result, math_equation = handle_calculate_tool_call(tool_call)
            messages.append(buffer)
            messages.append(response)
            image = generate_math_result_image(math_equation, result)
            if model == "GPT":   
                next_response = openai.chat.completions.create(model=GPT_MODEL, messages=messages, stream=True)
            else:
                next_response = gemini.chat.completions.create(model=GEMINI_MODEL, messages=messages, stream=True)
            for next_chunk in next_response:
                tool_answer += next_chunk.choices[0].delta.content or ""
                yield history + [{"role": "assistant", "content": tool_answer}], image

In [ ]:
def handle_calculate_tool_call(tool_call):
    arguments = json.loads(tool_call.function.arguments)
    math_equation = arguments.get('math_equation')
    result = calculate_math(math_equation)
    response = {
        "role": "tool",
        "content": json.dumps({"math_equation": math_equation, "result": result}),
        "tool_call_id": tool_call.id
    }
    return response, result, math_equation

In [ ]:
def transcribe(audio_file):
    if audio_file is None:
        return ""
    with open(audio_file, "rb") as f:
        transcription = openai.audio.transcriptions.create(
            model="gpt-4o-mini-transcribe", 
            file=f
        )
    return transcription.text

In [ ]:
with gradio.Blocks() as ui:
    with gradio.Row():
        chatbot = gradio.Chatbot(height=500, type="messages")
        image_output = gradio.Image(height=500)
    with gradio.Row():
        entry = gradio.Textbox(label="Chat with our code expert:")
        microphone = gradio.Audio(sources="microphone", type="filepath")
    with gradio.Row():
        ai_model = gradio.Dropdown(["GPT", "Gemini"], label="Select Model")
        clear = gradio.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history, None

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot, microphone]).then(
        chat, inputs=[chatbot, ai_model], outputs=[chatbot, image_output]
    )
    microphone.change(
        transcribe,
        inputs=[microphone],
        outputs=[entry]  
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch()